# 2. Solving the Workforce Allocation Problem with Constraint Programming (CP)

In the previous notebook, we introduced the Workforce Allocation Problem (WAP) and created a baseline solution using a simple greedy heuristic. While fast, that approach doesn't guarantee optimality. 

In this notebook, we'll formulate WAP as a **Constraint Programming (CP)** model. CP is a powerful paradigm for solving combinatorial optimization problems by focusing on constraints and variables. We will use it to find a provably optimal solution that minimizes the number of teams used.

### Prerequisites

This notebook requires the `discrete-optimization` library and its dependencies, including `ortools` and `cpmpy`.

In [ ]:
# On Colab: install the library
import sys

ON_COLAB = "google.colab" in sys.modules
if ON_COLAB:
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install discrete-optimization

### Imports

In [ ]:
from discrete_optimization.datasets import fetch_data_from_cp25
from discrete_optimization.generic_tools.cp_tools import ParametersCp

# Import problem and parser
from discrete_optimization.workforce.allocation.parser import (
    build_allocation_problem_from_scheduling,
)

# Import the CP-SAT solver
from discrete_optimization.workforce.allocation.solvers.cpsat import (
    CpsatTeamAllocationSolver,
    ModelisationAllocationOrtools,
)

# Import plotting utilities
from discrete_optimization.workforce.allocation.utils import plot_allocation_solution
from discrete_optimization.workforce.scheduling.parser import (
    get_data_available,
    parse_json_to_problem,
)

### Load a Problem Instance

For this problem, we'll load an instance from a JSON format that describes a more general scheduling problem. We then convert this into a `TeamAllocationProblem` instance.

In [ ]:
files = get_data_available()
if not files:
    fetch_data_from_cp25()
    files = get_data_available()

file_path = files[1]
scheduling_problem = parse_json_to_problem(file_path)
problem = build_allocation_problem_from_scheduling(problem=scheduling_problem)

## The CP Model Formulation

To solve the workforce allocation problem with Constraint Programming, we must define our problem in terms of variables, constraints, and an objective function.

### Decision Variables
The core decision is: *which team performs which task?* We can model this with binary variables:

$$ x_{i,j} = \begin{cases} 1 & \text{if task } i \text{ is assigned to team } j \\ 0 & \text{otherwise} \end{cases} $$

### Constraints
We translate the problem's rules into mathematical constraints:

1. **Assignment Constraint:** Each task must be assigned to exactly one team.
   $$ \sum_{j \in \text{Teams}} x_{i,j} = 1 \quad \forall i \in \text{Tasks} $$

2. **Conflict Constraint (coloring):** If tasks $i$ and $k$ overlap, they can't be assigned to the same team.
   $$ x_{i,j} + x_{k,j} \le 1 \quad \forall j \in \text{Teams}, \forall (i, k) \in \text{OverlappingTasks} $$
   Another way of modeling conflict constraint is to consider **cliques** of overlapping tasks, i.e set of tasks all overlapping with each other.
   In this case, $$\forall c \in Cliques, \forall j \in \text{Teams}, \sum_{i \in c}x_{i,j} \leq 1 $$, this generalizes the previous constraint.

3. **Compatibility/Availability Constraints:** 
A task can only be assigned to an allowed team. This is handled by only creating variables $x_{i,j}$ where team $j$ is allowed for task $i$.

### Objective Function
Our goal is to minimize the number of teams used. We introduce a binary variable $y_j$ for each team, which is $1$ if the team is used at all, and $0$ otherwise. 

* **Objective:**
    $$ \text{minimize} \sum_{j \in \text{Teams}} y_j $$

* **Linking Constraint:** The variable $y_j$ is forced to $1$ if any task is assigned to team $j$.
    $$ x_{i,j} \le y_j \quad \forall i \in \text{Tasks}, \forall j \in \text{Teams} $$

### Solving the Problem

Our library encapsulates this logic. We can instantiate the `CpsatTeamAllocationSolver`, build the model, and launch the solver.

In [ ]:
# Instantiate the solver
solver = CpsatTeamAllocationSolver(problem)

# Initialize the CP model with the binary variable formulation
solver.init_model(modelisation_allocation=ModelisationAllocationOrtools.BINARY)

# Set CP solver parameters (e.g., number of workers)
cp_params = ParametersCp.default_cpsat()
cp_params.nb_process = 6  # Use 6 cores

# Solve the problem with a 30-second time limit
result_storage = solver.solve(parameters_cp=cp_params, time_limit=30)

# Get the best solution found
optimal_solution, fitness = result_storage.get_best_solution_fit()

In [ ]:
print(f"CP Solver Finished:")
print(f"  - Status: {solver.status_solver}")
print(f"  - Is the solution feasible? {problem.satisfy(optimal_solution)}")
print(f"  - Optimal number of teams used: {int(optimal_solution.kpis['nb_teams'])}")

Compare this result to the greedy solution from the previous notebook. The CP solver should find a solution that uses an equal or fewer number of teams, demonstrating the power of optimization over simple heuristics.

### Visualizing the Optimal Solution

In [ ]:
fig = plot_allocation_solution(
    problem=problem,
    sol=optimal_solution,
    title="Optimal Allocation Schedule (CP-SAT)",
    display=False,
)
fig.show()

## Conclusion

In this notebook, we successfully modeled the Workforce Allocation Problem using Constraint Programming and found an optimal solution. This demonstrates how a declarative approach like CP can handle complex combinatorial constraints effectively.

But what happens when the real world intervenes? A team might call in sick, their equipment might break, or their availability might change. A static, optimal plan can quickly become invalid.

➡️ **Next up:** In [Notebook 3](3_Disruptions_and_Scenarios.ipynb), we'll explore how to model these disruptions and see what happens to our carefully crafted solution.